In [ ]:
# del session.logger
# del session

import multiprocessing

from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
from config import SpectralConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SpectralConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type="fourier",
    merger_emb_dim=2048,
    merger_channels=256,
    merger_dropout=True,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    cnn_channels=[384, 384, 384, 384, 384, 384],
    kernel_size=3,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=True,
    bins=16,
    hop_length=2,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

config.batch_size = 64
config.learning_rate = 3e-4
config.new_freq = 200

config.brain_encoder_config.half = False
config.brain_encoder_config.bins = 32
config.brain_encoder_config.hop_length = 2
config.brain_encoder_config.cnn_channels = [384, 256, 128, 64, 32, 12]

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/spectrogram/200hz_32bins",
    clear_cache=False,
    cache_name="cache/2",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SpectralConv initialized with 3464400 parameters, cond: ['study', 'subject']
Merger False, merger channels 256
ConvBlocks: [384, 256, 128, 64, 32, 12], hidden_dim: 384, params 2989616


2025-01-20 14:23:49,475	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [05:30<00:00,  2.45s/it]


Epoch 1 completed. Loss: 4440777.6797, Clip Loss: 100.9670, MSE Loss: 11101792.7413
Accuracy: 0.0162, Top 5: 0.0822, Top 10: 0.1655
Test unseen_subject completed. Accuracy: 0.0195, Top 5: 0.1015, Top 10: 0.1905
Test unseen_task completed. Accuracy: 0.0165, Top 5: 0.0832, Top 10: 0.1629
Test unseen_both completed. Accuracy: 0.0150, Top 5: 0.0975, Top 10: 0.1875
Testing completed in 0.49m.
Epoch 1 completed in 6.02m. 0.04m per recording.


Training Epoch 2:   0%|          | 0/135 [00:00<?, ?it/s]

In [ ]:
del session.logger
del session.model
del session

import multiprocessing

from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
from config import SpectralConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SpectralConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type="fourier",
    merger_emb_dim=2048,
    merger_channels=256,
    merger_dropout=True,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    cnn_channels=[384, 384, 384, 384, 384, 384],
    kernel_size=3,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=True,
    bins=16,
    hop_length=2,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

config.batch_size = 64
config.learning_rate = 3e-4
config.new_freq = 200

config.brain_encoder_config.half = False
config.brain_encoder_config.bins = 16
config.brain_encoder_config.hop_length = 2
config.brain_encoder_config.cnn_channels = [384, 256, 128, 64, 32, 24]

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/spectrogram/200hz_16bins",
    clear_cache=False,
    cache_name="cache/2",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

In [ ]:
# del session.logger
# del session.model
# del session

# import multiprocessing

# from train.training_session_v0 import TrainingSessionV0
# from config import TrainingConfigV0
# from config import SpectralConvConfig

# data_partition = {
#     "gwilliams2023": {
#         "testing_subjects": [19, 20, 21],
#         "testing_tasks": [0],
#     },
#     # "armeini2022": {
#     #     "testing_subjects": [],
#     #     "testing_tasks": [8, 9],
#     # },
# }

# model_config = SpectralConvConfig(
#     # Str to list of possible conditions
#     mel_normalization=False,
#     conditions={
#         "study": [],
#         "subject": [],
#     },
#     # Channels
#     in_channels=208,
#     out_channels=128,
#     hidden_dim=384,
#     dropout=0.2,
#     initial_batch_norm=True,
#     # Sensor layout settings
#     layout_dim=2,
#     layout_proj=True,
#     layout_scaling="minmax",
#     # Merger with spatial attn
#     merger=False,
#     merger_emb_type="fourier",
#     merger_emb_dim=2048,
#     merger_channels=256,
#     merger_dropout=True,
#     merger_conditional=None,
#     # Inital
#     initial_linear=384,
#     initial_depth=1,
#     # Conditional layers
#     conditional_layers=False,
#     conditional_layers_dim=None,  # input or hidden_dim
#     # Conv layer overall structure
#     cnn_channels=[384, 384, 384, 384, 384, 384],
#     kernel_size=3,
#     dilation_growth=2,
#     dilation_period=5,
#     glu=1,
#     conv_dropout=0.2,
#     dropout_input=0.2,
#     batch_norm=True,
#     half=True,
#     cnn_pos_encoding=True,
#     bins=16,
#     hop_length=2,
#     # Quantizer
#     quantizer=False,
#     num_codebooks=0,
#     codebook_size=0,
#     quantizer_commitment=0,
#     quantizer_temp_init=0,
#     quantizer_temp_min=0,
#     quantizer_temp_decay=0,
#     # Transformers Encoders
#     transformer_input=None,
#     transformer_encoder_emb=None,
#     transformer_encoder_layers=0,
#     transformer_encoder_heads=0,
#     # Transformer Decoders
#     transformer_decoder_emb=None,
#     transformer_decoder_layers=0,
#     transformer_decoder_heads=0,
#     transformer_decoder_dim=0,
# )

# config = TrainingConfigV0(
#     brain_encoder_config=model_config,
#     data_partition=data_partition,
#     # Pre-processing parameters
#     # Brain
#     new_freq=200,
#     frequency_bands={"all": (0.5, 80)},
#     max_random_shift=1.0,
#     window_size=4,
#     window_stride=1,
#     brain_clipping=None,
#     baseline_window=0.5,
#     notch_filter=True,
#     scaling="both",
#     delay=0.15,
#     # Hyperparameters
#     learning_rate=3e-4,
#     weight_decay=1e-4,
#     epochs=50,
#     batch_size=256,
#     use_clip_loss=True,
#     use_mse_loss=True,
#     alpha=0.6,
#     random_test_size=10,
#     seed=42,
# )

# config.batch_size = 128
# config.learning_rate = 3e-4
# config.new_freq = 1000

# config.brain_encoder_config.half = False
# config.brain_encoder_config.bins = 64
# config.brain_encoder_config.hop_length = 10
# config.brain_encoder_config.cnn_channels = [384, 256, 128, 64, 32, 6]

# session = TrainingSessionV0(
#     config=config,
#     studies={study: "audio" for study in data_partition.keys()},
#     data_path="data",
#     save_path="saves/phase1/spectrogram/1000hz_64bins",
#     clear_cache=False,
#     cache_name="cache/1",
# )

# try:
#     session.train(
#         device="cuda",
#         buffer_size=30,
#         num_workers=(multiprocessing.cpu_count() - 2),
#         max_cache_size=400,
#         current_epoch=0,
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SpectralConv initialized with 3460544 parameters, cond: ['study', 'subject']
Merger False, merger channels 256
ConvBlocks: [384, 256, 128, 64, 32, 6], hidden_dim: 384, params 2985760


Training Epoch 1: 100%|██████████| 135/135 [23:57<00:00, 10.65s/it] 


(raylet) Spilled 4389 MiB, 3 objects, write throughput 707 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 17809 MiB, 9 objects, write throughput 881 MiB/s.
(raylet) Spilled 21594 MiB, 10 objects, write throughput 750 MiB/s.
(raylet) Spilled 31702 MiB, 13 objects, write throughput 964 MiB/s.
(raylet) Spilled 35480 MiB, 14 objects, write throughput 951 MiB/s.
(raylet) Spilled 71375 MiB, 26 objects, write throughput 1122 MiB/s.
(raylet) Spilled 144310 MiB, 56 objects, write throughput 1219 MiB/s.


Epoch 1 completed. Loss: 15.8955, Clip Loss: 7.6429, MSE Loss: 28.2745
Accuracy: 0.0085, Top 5: 0.0428, Top 10: 0.0858
Test unseen_subject completed. Accuracy: 0.0100, Top 5: 0.0540, Top 10: 0.1070
Test unseen_task completed. Accuracy: 0.0079, Top 5: 0.0423, Top 10: 0.0828
Test unseen_both completed. Accuracy: 0.0150, Top 5: 0.0512, Top 10: 0.1087
Testing completed in 3.56m.
Epoch 1 completed in 27.52m. 0.20m per recording.


Training Epoch 2:  10%|▉         | 13/135 [02:27<07:56,  3.91s/it] 

In [ ]:
# del session.logger
# del session.model
# del session

# import multiprocessing

# from train.training_session_v0 import TrainingSessionV0
# from config import TrainingConfigV0
# from config import SpectralConvConfig

# data_partition = {
#     "gwilliams2023": {
#         "testing_subjects": [19, 20, 21],
#         "testing_tasks": [0],
#     },
#     # "armeini2022": {
#     #     "testing_subjects": [],
#     #     "testing_tasks": [8, 9],
#     # },
# }

# model_config = SpectralConvConfig(
#     # Str to list of possible conditions
#     mel_normalization=False,
#     conditions={
#         "study": [],
#         "subject": [],
#     },
#     # Channels
#     in_channels=208,
#     out_channels=128,
#     hidden_dim=384,
#     dropout=0.2,
#     initial_batch_norm=True,
#     # Sensor layout settings
#     layout_dim=2,
#     layout_proj=True,
#     layout_scaling="minmax",
#     # Merger with spatial attn
#     merger=False,
#     merger_emb_type="fourier",
#     merger_emb_dim=2048,
#     merger_channels=256,
#     merger_dropout=True,
#     merger_conditional=None,
#     # Inital
#     initial_linear=384,
#     initial_depth=1,
#     # Conditional layers
#     conditional_layers=False,
#     conditional_layers_dim=None,  # input or hidden_dim
#     # Conv layer overall structure
#     cnn_channels=[384, 384, 384, 384, 384, 384],
#     kernel_size=3,
#     dilation_growth=2,
#     dilation_period=5,
#     glu=1,
#     conv_dropout=0.2,
#     dropout_input=0.2,
#     batch_norm=True,
#     half=True,
#     cnn_pos_encoding=True,
#     bins=16,
#     hop_length=2,
#     # Quantizer
#     quantizer=False,
#     num_codebooks=0,
#     codebook_size=0,
#     quantizer_commitment=0,
#     quantizer_temp_init=0,
#     quantizer_temp_min=0,
#     quantizer_temp_decay=0,
#     # Transformers Encoders
#     transformer_input=None,
#     transformer_encoder_emb=None,
#     transformer_encoder_layers=0,
#     transformer_encoder_heads=0,
#     # Transformer Decoders
#     transformer_decoder_emb=None,
#     transformer_decoder_layers=0,
#     transformer_decoder_heads=0,
#     transformer_decoder_dim=0,
# )

# config = TrainingConfigV0(
#     brain_encoder_config=model_config,
#     data_partition=data_partition,
#     # Pre-processing parameters
#     # Brain
#     new_freq=200,
#     frequency_bands={"all": (0.5, 80)},
#     max_random_shift=1.0,
#     window_size=4,
#     window_stride=1,
#     brain_clipping=None,
#     baseline_window=0.5,
#     notch_filter=True,
#     scaling="both",
#     delay=0.15,
#     # Hyperparameters
#     learning_rate=3e-4,
#     weight_decay=1e-4,
#     epochs=50,
#     batch_size=256,
#     use_clip_loss=True,
#     use_mse_loss=True,
#     alpha=0.6,
#     random_test_size=10,
#     seed=42,
# )

# config.batch_size = 256
# config.learning_rate = 3e-4
# config.new_freq = 1000

# config.brain_encoder_config.half = False
# config.brain_encoder_config.bins = 32
# config.brain_encoder_config.hop_length = 10
# config.brain_encoder_config.cnn_channels = [384, 256, 128, 64, 32, 12]

# session = TrainingSessionV0(
#     config=config,
#     studies={study: "audio" for study in data_partition.keys()},
#     data_path="data",
#     save_path="saves/phase1/spectrogram/1000hz_32bins",
#     clear_cache=False,
#     cache_name="cache/1",
# )

# try:
#     session.train(
#         device="cuda",
#         buffer_size=30,
#         num_workers=(multiprocessing.cpu_count() - 2),
#         max_cache_size=400,
#         current_epoch=0,
#     )
# except KeyboardInterrupt as e:
#     print("Exited")